<a href="https://colab.research.google.com/github/nithyap2209/Leaf-Disease-Detection/blob/main/leaf_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'leaf-disease-detection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2272471%2F3814063%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T103130Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D032c095998076e11733a2eb52d047fd74b84c6cbfc6efc3b2b3c08b76f3800aa67ab10001ce4325586b36c65820de1115ec427f21c6cfcf91a02248e92938509b0f2d370bc898c8cbba9885b7589d6990b930e539c0bdda63289693ed965e393ad8002ee57dc088971dc343b2f0a4c200a95cfb9c18a3d1993b730d28e6b64a1c2140558eed91154c0f9b429fb4782f387ba7076b0921b1a33239d2835fe05aef2c58d3e90207b1ec7a5835d7abfbcf5c6086e3d466f43b390ad8130c767483038173ecf62a8617361035ad762593997daa05ae74dd2e98542c5e8be4236b0fab429f6af662f06510bc6096d68bc480873776c6d734d4d493c9fe52d145436b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1437121598 bytes downloaded
Downloaded and uncompressed: leaf-disease-detection-dataset
Data source import complete.


In [38]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
# Define data directories
train_dir = '/kaggle/input/leaf-disease-detection-dataset/dataset/train'
test_dir = '/kaggle/input/leaf-disease-detection-dataset/dataset/test'


In [40]:
# Define model parameters
input_shape = (128, 128, 3)  # Adjust image dimensions as needed
num_classes = len(os.listdir(train_dir))
batch_size = 32
epochs = 10

In [41]:
# Create a CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [44]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Epoch 1/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 2572s 1s/step - accuracy: 0.3027 - loss: 2.4966 - val_accuracy: 0.7241 - val_loss: 0.9361
Epoch 2/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 43s 19ms/step - accuracy: 0.6875 - loss: 0.8994 - val_accuracy: 1.0000 - val_loss: 0.1546
Epoch 3/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 2478s 1s/step - accuracy: 0.6516 - loss: 1.1299 - val_accuracy: 0.7939 - val_loss: 0.6410
Epoch 4/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1s 37us/step - accuracy: 0.7188 - loss: 0.7165 - val_accuracy: 1.0000 - val_loss: 0.0409
Epoch 5/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 2435s 1s/step - accuracy: 0.7344 - loss: 0.8435 - val_accuracy: 0.8778 - val_loss: 0.3775
Epoch 6/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1s 41us/step - accuracy: 0.7812 - loss: 0.7122 - val_accuracy: 1.0000 - val_loss: 0.0545
Epoch 7/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 2486s 1s/step - accuracy: 0.7736 - loss: 0.7202 - val_accuracy: 0.8882 - val_loss: 0.3475
Epoch 8/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1s 33us/step - accuracy: 0.9062 - lo

In [45]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

550/550 ━━━━━━━━━━━━━━━━━━━━ 162s 295ms/step - accuracy: 0.9123 - loss: 0.2739
Test accuracy: 91.08%


In [46]:
import json

# Save class names to a file
class_names = list(train_generator.class_indices.keys())
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

# Save the model
model.save('leaf_disease_model.h5')


In [47]:
! pip install streamlit -q

In [48]:
!wget -q -O - ipv4.icanhazip.com

34.86.113.56


In [49]:
# !pip install streamlit pyngrok


In [50]:
%%writefile leaf.py
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
import json

# Load the trained model
model = tf.keras.models.load_model('leaf_disease_model.h5')

# Load class names from the JSON file
with open('class_names.json', 'r') as f:
    class_names = json.load(f)

# Define image preprocessing function
def preprocess_image(image):
    image = image.resize((128, 128))  # Resize image to match model input size
    image_array = np.array(image) / 255.0  # Normalize image
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    return image_array

# Define Streamlit app
st.title("Leaf Disease Detection")
st.write("Upload an image of a leaf to classify its disease.")

# Image upload widget
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the image
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Preprocess the image
    image_array = preprocess_image(image)

    # Predict the class of the image
    predictions = model.predict(image_array)
    class_index = np.argmax(predictions[0])

    # Check if class_index is within the range of available class names
    if class_index < len(class_names):
        class_name = class_names[class_index]
        # Display the result
        st.write(f"Prediction: {class_name}")
        st.write(f"Confidence: {predictions[0][class_index] * 100:.2f}%")
    else:
        st.write("Error: The model's class index is out of range.")


Overwriting leaf.py


In [ ]:
#pip install -r requirements.txt


In [ ]:
# !pip install --upgrade pyngrok


In [ ]:
from pyngrok import ngrok

# Replace 'your_authtoken' with your actual Ngrok authtoken
ngrok.set_auth_token("2k4N3yToOiU2PRzuUnSoCoQfwXs_88KGceXEeQ864w7ABLECz")

# Set up a tunnel to the Streamlit app
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at {public_url}")

# Run the Streamlit app
!streamlit run leaf.py --server.port 8501


Streamlit app is live at NgrokTunnel: "https://bf7f-34-86-113-56.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.113.56:8501

2024-08-02 10:48:04.629918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 10:48:04.968033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 10:48:05.066063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 10:48:08.624710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38